In [55]:
!pip install Mastodon.py
!pip install aiohttp

In [56]:
from mastodon import Mastodon
from bs4 import BeautifulSoup
import json
import os
import threading
import time

#folder_path = '/colab_json_files/' 
#if not os.path.exists(folder_path):
#    os.makedirs(folder_path)

In [57]:
mastodon = Mastodon(access_token='YvMBnWgknUL41MI3QjRaOn7goLZ1rRpgyHWZjrxIF9E', api_base_url='https://mastodon.social')

In [41]:
def handle_rate_limiting(mastodon):
    remaining_requests = mastodon.ratelimit_remaining
    reset_time = mastodon.ratelimit_reset
    current_time = time.time()

    if remaining_requests == 0 and reset_time:
        sleep_duration = reset_time - current_time + 1
        if sleep_duration > 0:
            print(f"Rate limit reached. Sleeping for {sleep_duration:.2f} seconds.")
            time.sleep(sleep_duration)

In [59]:
hashtags = ['trump', 'donaldtrump', 'kamalaharris', 'harris'] #hashtags I used for search
#hashtags = ['donaldtrump'] 
data_json = dict()

In [60]:
def get_text(content):
  soup = BeautifulSoup(content, 'html.parser')
  return soup.get_text()

In [61]:
def get_replies_and_ids(id, data, hashtag):
    dt = []
    context = mastodon.status_context(id)
    handle_rate_limiting(mastodon)
    replies = context['descendants']
    
    for reply in replies:
        reply_data = {
        'id': reply['id'],
        'content': get_text(reply['content']),
        'replies': [],
        'toxicity': 0,
        'hashtag': hashtag
        }
        descendant_replies = get_replies_and_ids(reply['id'], data, hashtag)
        if descendant_replies:
          reply_data['replies'].extend(descendant_replies)
        data[reply['id']] = reply_data
        dt.append(reply['id'])
    return dt

In [8]:
#def get_posts_and_replies(hashtag, limit=130, page_limit=20):
#    print("Starting collecting data for hashtag #" + hashtag)
#    data = dict()
#    #posts = mastodon.timeline_hashtag(hashtag, limit=limit)
#    max_id = None
#    total_collected = 0
#
#    while total_collected < limit:
#        print(f"Starting collecting from {total_collected} posts for hashtag #{hashtag}")
#        batch_limit = min(page_limit, limit-total_collected)
#        posts = mastodon.timeline_hashtag(hashtag, limit=batch_limit, max_id=max_id)
#
#        if not posts:
#            print(f"No more posts to fetch for hashtag #{hashtag}")
#            break
#
#        for post in posts:
#            post_data = {
#            'id': post['id'],
#            'content': get_text(post['content']),
#            'replies': [],
#            'toxicity': 0,
#            'hashtag': hashtag
#            }
#            post_data['replies'].extend(get_replies_and_ids(post['id'], data, hashtag))
#            data[post['id']] = post_data
#
#        max_id = posts[-1]['id']
#        total_collected += len(posts)
#        print(f"Collected {total_collected} posts for hashtag #{hashtag}")
#    print("Collected data from hashtag #" + hashtag)
#    return data

In [62]:
def get_posts_and_replies(hashtag, limit=130, page_limit=40):
    print("Starting collecting data for hashtag #" + hashtag)
    data = dict()
    max_id = None
    total_collected = 0

    while total_collected < limit:
        print(f"Starting collecting from {total_collected} posts for hashtag #{hashtag}")
        batch_limit = min(page_limit, limit - total_collected)
        posts = mastodon.timeline_hashtag(hashtag, limit=batch_limit, max_id=max_id)
        handle_rate_limiting(mastodon)

        if not posts:
            print(f"No more posts to fetch for hashtag #{hashtag}")
            break

        for post in posts:
            post_data = {
                'id': post['id'],
                'content': get_text(post['content']),
                'replies': [],
                'toxicity': 0,
                'hashtag': hashtag
            }
            post_data['replies'].extend(get_replies_and_ids(post['id'], data, hashtag))
            data[post['id']] = post_data

        max_id = posts[-1]['id']
        total_collected += len(posts)
        print(f"Collected {total_collected} posts for hashtag #{hashtag}")

    print("Collected data from hashtag #" + hashtag)
    return data

In [46]:
for hashtag in hashtags:
  data_json = get_posts_and_replies(hashtag)
  with open(f'{hashtag}.json', 'w') as f:
    json.dump(data_json, f, indent=4)

Starting collecting data for hashtag #donaldtrump
Starting collecting from 0 posts for hashtag #donaldtrump
Collected 40 posts for hashtag #donaldtrump
Collected data from hashtag #donaldtrump


In [63]:
def analyze_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        entry_count = len(data)
        print(f"File: {os.path.basename(file_path)}")
        print(f"Number of entries: {entry_count}")

        if entry_count > 0:
            sample_id = next(iter(data))
            sample_data = data[sample_id]
            print(f"Sample entry - ID: {sample_id}")
            if isinstance(sample_data, dict):
                print(f"Sample data keys: {', '.join(sample_data.keys())}")
    finally:
        return

def count_contents(target_files):
    for file_name in target_files:
        if os.path.exists(file_name):
            analyze_json_file(file_name)
        else:
            print(f"File not found: {file_name}")
            print("-" * 30)

jsons = ['trump.json', 'harris.json', 'donaldtrump.json', 'kamalaharris.json']
count_contents(jsons)

File: trump.json
Number of entries: 269
Sample entry - ID: 113301683458757153
Sample data keys: id, content, replies, toxicity, hashtag
File: harris.json
Number of entries: 307
Sample entry - ID: 113299532808803896
Sample data keys: id, content, replies, toxicity, hashtag
File: donaldtrump.json
Number of entries: 206
Sample entry - ID: 113301790869024476
Sample data keys: id, content, replies, toxicity, hashtag
File: kamalaharris.json
Number of entries: 206
Sample entry - ID: 113299439517108438
Sample data keys: id, content, replies, toxicity, hashtag


In [31]:
def combine_json(hashtags):
    combined_data = {}

    for hashtag in hashtags:
        file = f'{hashtag}.json'
        if os.path.exists(file):
            with open(file, 'r') as f:
                data = json.load(f)
            for post_id, post_data in data.items():
                if post_id in combined_data:
                    combined_data[post_id]['hashtag'].append(hashtag)
                else:
                    combined_data[post_id] = post_data
                    combined_data[post_id]['hashtag'] = [hashtag]
        else:
            print(f'File {file}.json not found')
            
    return combined_data

def save_json(file_name, data):
    with open(file_name, 'w') as f:
        json.dump(data, f, indent=4)

In [32]:
hashtags = ['trump', 'harris', 'donaldtrump', 'kamalaharris']
combined_data = combine_json(hashtags)
file_name = 'data.json'
save_json(file_name, combined_data)

In [34]:
print('Total number of posts and replies:', len(combined_data))
analyze_json_file(file_name)

932
File: data.json
Number of entries: 932
Sample entry - ID: 113301683458757153
Sample data keys: id, content, replies, toxicity, hashtag
